In [2]:
pip install -q sagemaker-experiments

Keyring is skipped due to an exception: 'keyring.backends'


In [3]:
import sagemaker
import json
import boto3

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = 'module_5/part_1'

In [4]:
import numpy as np
import pandas as pd
import os

In [5]:
source_file='s3://sagemaker-sample-files/datasets/tabular/synthetic/churn.txt'
local_prefix='churn_data'
os.makedirs(local_prefix, exist_ok=True)
sagemaker.s3.S3Downloader.download(source_file, local_prefix)

In [7]:
df=pd.read_csv(f'./{local_prefix}/churn.txt')
df['CustomerID']=df.index
df

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?,CustomerID
0,PA,163,806,403-2562,no,yes,300,8.162204,3,7.579174,...,6.508639,4.065759,100,5.111624,4.928160,6,5.673203,3,True.,0
1,SC,15,836,158-8416,yes,no,0,10.018993,4,4.226289,...,9.972592,7.141040,200,6.436188,3.221748,6,2.559749,8,False.,1
2,MO,131,777,896-6253,no,yes,300,4.708490,3,4.768160,...,4.566715,5.363235,100,5.142451,7.139023,2,6.254157,4,False.,2
3,WY,75,878,817-5729,yes,yes,700,1.268734,3,2.567642,...,2.333624,3.773586,450,3.814413,2.245779,6,1.080692,6,False.,3
4,WY,146,878,450-4942,yes,no,0,2.696177,3,5.908916,...,3.670408,3.751673,250,2.796812,6.905545,4,7.134343,6,True.,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,NH,4,787,151-3162,yes,yes,800,10.862632,5,7.250969,...,8.026482,4.921314,350,6.748489,4.872570,8,2.122530,9,False.,4995
4996,SD,140,836,351-5993,no,no,0,1.581127,8,3.758307,...,1.328827,0.939932,300,4.522661,6.938571,2,4.600473,4,False.,4996
4997,SC,32,836,370-3127,no,yes,700,0.163836,5,4.243980,...,2.340554,0.939469,450,5.157898,4.388328,7,1.060340,6,False.,4997
4998,MA,142,776,604-2108,yes,yes,600,2.034454,5,3.014859,...,3.470372,6.076043,150,4.362780,7.173376,3,4.871900,7,True.,4998


In [8]:
df.columns

Index(['State', 'Account Length', 'Area Code', 'Phone', 'Int'l Plan',
       'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge',
       'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
       'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls', 'Churn?', 'CustomerID'],
      dtype='object')

In [9]:
df[["Int'l Plan", "VMail Plan"]] = df[["Int'l Plan", "VMail Plan"]].replace(to_replace=['yes', 'no'], value=[1, 0])

In [10]:
df['Churn?'] = df['Churn?'].replace(to_replace=['True.', 'False.'], value=[1, 0])

In [11]:
columns=['Churn?', 'State', 'Account Length', "Int'l Plan",
           'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge',
           'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
           'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
           'CustServ Calls']
df.index = df['CustomerID']
df_processed = df[columns]

In [12]:
df_processed.head()

,Churn?,State,Account Length,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls
CustomerID,,,,,,,,,,,,,,,,,,,
0,1,PA,163,0,1,300,8.162204,3,7.579174,3.933035,4,6.508639,4.065759,100,5.111624,4.928160,6,5.673203,3
1,0,SC,15,1,0,0,10.018993,4,4.226289,2.325005,0,9.972592,7.141040,200,6.436188,3.221748,6,2.559749,8
2,0,MO,131,0,1,300,4.708490,3,4.768160,4.537466,3,4.566715,5.363235,100,5.142451,7.139023,2,6.254157,4
3,0,WY,75,1,1,700,1.268734,3,2.567642,2.528748,5,2.333624,3.773586,450,3.814413,2.245779,6,1.080692,6
4,1,WY,146,1,0,0,2.696177,3,5.908916,6.015337,3,3.670408,3.751673,250,2.796812,6.905545,4,7.134343,6


In [13]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_processed, test_size=0.1, random_state=42, 
                                     shuffle=True, stratify=df_processed['State'])

In [ ]:
from sagemaker.amazon.amazon_estimator import image_uris
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from botocore.exceptions import ClientError
import time
from time import gmtime, strftime

image = image_uris.retrieve(region=region, framework='xgboost', version='1.3-1')
train_instance_type = 'ml.m5.xlarge'
train_instance_count = 1
s3_output = f's3://{bucket}/{prefix}/{local_prefix}/training'

experiment_name = 'churn-prediction'

try:
    experiment = Experiment.create(
        experiment_name=experiment_name, 
        description='Training churn prediction models based on telco churn dataset.')
except ClientError as e:
    print(f'{experiment_name} experiment already exists! Reusing the existing experiment.')
    

def launch_training_job(state, train_data_s3, val_data_s3):
    exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
    jobname = f'churn-xgb-{state}-{exp_datetime}'

    # Creating a new trial for the experiment
    exp_trial = Trial.create(experiment_name=experiment_name, 
                             trial_name=jobname)

    experiment_config={
        'ExperimentName': experiment_name,
        'TrialName': exp_trial.trial_name,
        'TrialComponentDisplayName': 'Training'}

    xgb = sagemaker.estimator.Estimator(image,
                                        role,
                                        instance_count=train_instance_count,
                                        instance_type=train_instance_type,
                                        output_path=s3_output,
                                        enable_sagemaker_metrics=True,
                                        sagemaker_session=sess)
    xgb.set_hyperparameters(objective='binary:logistic', num_round=20)
    
    train_input = sagemaker.inputs.TrainingInput(s3_data=train_data_s3, 
                                                 content_type='csv')
    val_input = sagemaker.inputs.TrainingInput(s3_data=val_data_s3, 
                                               content_type='csv')
    data_channels={'train': train_input, 'validation': val_input}
    
    xgb.fit(inputs=data_channels, job_name=jobname, 
            experiment_config=experiment_config, wait=False)

    return xgb

In [ ]:
columns_no_target=['Account Length', "Int'l Plan", 'VMail Plan', 'VMail Message', 'Day Mins',
                   'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
                   'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge', 'CustServ Calls']

df_test.to_csv(f'{local_prefix}/churn_test.csv')
df_test[columns_no_target].to_csv(f'{local_prefix}/churn_test_no_target.csv', 
                                  index=False)

sagemaker.s3.S3Uploader.upload(f'{local_prefix}/churn_test.csv', 
                               f's3://{bucket}/{prefix}/{local_prefix}')
sagemaker.s3.S3Uploader.upload(f'{local_prefix}/churn_test_no_target.csv', 
                               f's3://{bucket}/{prefix}/{local_prefix}')